In [13]:
import requests
import json
class ProdSysApi:
    
    def __init__(self, url_list, url_single, user, password):
        self._auth = requests.auth.HTTPBasicAuth(user, password)
        self._urls = [url_single, url_list]
        
    def api_url(self, id=None):
        if id is None:
            return self._urls[1]
        else:
            return self._urls[0].format(id)
    
    def list(self):
        response = requests.get(self.api_url(), auth=self._auth)
        print(len(response.content))
        data = response.json()
        return data
        
    def get(self, id):
        response = requests.get(self.api_url(id), auth=self._auth)
        data = response.json()
        return data
    
    def new(self, data):
            return requests.post(
                self.api_url(), 
                auth = self._auth,
                json = data,
            )
        
    def delete(self, id):
            return requests.delete(
                self.api_url(), 
                auth = self._auth,
            )
        
    def patch(self, id, data):
        if 'api/indesign/' in self._urls[0]:
            return requests.post(
                self.api_url(id), 
                auth = self._auth,
                data = {'json': json.dumps(data)},
            )
        else:
            return requests.patch(
                self.api_url(id), 
                auth = self._auth,
                json = data,
            )
            

        
old_api = ProdSysApi(
    user='api',
    password='Avier1gi',
    url_list = 'http://universitas.no/admin/api/indesign/json/',
    url_single = 'http://universitas.no/admin/api/indesign/{}/json/',
)
new_api = ProdSysApi(
    user='api',
    password='Avier1gi',
    url_list = 'http://universitas.no/api/legacy/?limit=500',
    url_single = 'http://universitas.no/api/legacy/{}/',
)
local_api = ProdSysApi(
    user='api',
    password='api',
    url_list = 'http://web:8000/api/legacy/?limit=10',
    url_single = 'http://web:8000/api/legacy/{}/',
)



In [24]:
prodsaker = [old_api.get(ps['prodsak_id']) for ps in old_api.list() if ps['produsert'] < '8']


407940


In [25]:
len(prodsaker)

81

In [30]:
# Remove work in progress stories
qs = Story.objects.filter(publication_status__lte=Story.STATUS_FROM_DESK)
qs.delete()


def pre_save(model_instance, add):
    """ Returns field's value just before saving. """
    return getattr(model_instance, 'modified')

def update_timestamps(story, created, modified):
    story.created = created 
    story.modified = modified
    story._meta.get_field('modified').pre_save = pre_save
    story.save()


import pytz
def get_status(produsert, flagg, **kwargs):
    flag_status = {
        1: Story.STATUS_JOURNALIST,
        2: Story.STATUS_SUBEDITOR,
        3: Story.STATUS_EDITOR,
        4: Story.STATUS_EDITOR
    }.get(int(flagg))
    status = {
        Prodsak.DRAFT: flag_status,
        Prodsak.OLD_DRAFT: flag_status,
        Prodsak.READY_FOR_PRINT: Story.STATUS_TO_DESK,
        Prodsak.IMPORTED_TO_INDESIGN: Story.STATUS_AT_DESK,
        Prodsak.EXPORTED_FROM_INDESIGN: Story.STATUS_FROM_DESK,
    }.get(int(produsert))
    return status


def timeparser(value):
    print(value)
    dt=pytz.datetime.datetime.strptime(value, '%Y-%m-%d %H:%M:%S')
    return pytz.timezone('Europe/Oslo').localize(dt)
# for prodsak in filter(lambda s: len(s['mappe']) == 2, prodsaker):

for ps in prodsaker:
    ps = dict(**ps)
    ps['produsert'] = get_status(**ps)
    if ps['mappe'] == '\ufffdr':
        ps['mappe'] = 'år'
    response = local_api.new(ps)
        
    print('{prodsak_id} {mappe} {arbeidstittel} {response.status_code}'.format(**ps, response=response))
    if response.status_code == 201:
        data = response.json()
        created = timeparser(ps.get('dato'))
        modified = timeparser(ps.get('version_date'))
        diff = modified-created
        story = Story.objects.get(pk=data['prodsak_id'])
        update_timestamps(story, created, modified)
        story.refresh_from_db()
        diff2 = story.modified - story.created
        print(diff, diff==diff2)
    elif response.status_code == 400:
        ...
    else:
        print(response.status_code)
        print(json.dumps(ps, ensure_ascii=False, indent=2))
        error = response.text.split('Traceback (most recent call last):')[-1].strip()
        print(error.splitlines()[-1])
        print(error)
        qs = Story.objects.filter(publication_status__lt=Story.STATUS_FROM_DESK)
        qs.delete()
        break

23260 vispor 17 Vi spør 201
2017-08-22 21:03:51
2017-08-23 00:59:48
3:55:57 True
23257 quiz 17 Quiz 201
2017-08-22 18:30:23
2017-08-23 00:59:48
6:29:25 True
23252 nyhetsnotis 17 NYH NOTISER 201
2017-08-22 20:30:31
2017-08-22 20:30:31
0:00:00 True
23192 nyhet 16 nyh karaktersak ekspert 201
2017-08-14 10:15:55
2017-08-22 22:10:36
8 days, 11:54:41 True
23214 nyhet 16 nyh Fem på hookern 400
23218 nyhet 16 notiser 201
2017-08-15 18:26:36
2017-08-16 00:44:18
6:17:42 True
23212 nyhet Hooke-nekt ved HiOA 201
2017-08-15 20:44:33
2017-08-15 22:46:18
2:01:45 True
22587 nyhet ISS oppfølger 17 201
2017-08-22 19:52:22
2017-08-22 23:45:02
3:52:40 True
23255 nyhet Svindyr faddermorro 201
2017-08-22 22:53:17
2017-08-23 00:26:20
1:33:03 True
23174 magasin MAGCurlingbarn på Blindern-isen 201
2017-08-15 14:55:48
2017-08-15 14:55:48
0:00:00 True
23225 magasin Tips til ferske studenter 201
2017-08-15 23:01:52
2017-08-15 23:38:11
0:36:19 True
23223 leder 16 Leder  201
2017-08-15 21:47:30
2017-08-15 21:47:30


In [31]:
prodsaker[0]

{'arbeidstittel': '17 Vi spør',
 'dato': '2017-08-22 21:03:51',
 'flagg': '4',
 'id': '156366',
 'journalist': 'BTH',
 'kommentar': 'Vellykket eksport fra InDesign!  Endring: 18 tegn\nWednesday, August 23 2017 00:59:48\nnull',
 'mappe': 'vispor',
 'prodsak_id': '23260',
 'produsert': '6',
 'tekst': '@tit:HF-guttens hevn\r\n\r\n@ing:Til tross for at hun kaller seg debattredaktør og «forkjemper for det frie ord,» gjorde Runa Fjellanger alt som sto i hennes makt for å stoppe tilsvaret fra en snurt HF-gutt.\r\n\r\n@txt:-- <I>Hei Runa! Har fått inn et temmelig krast leserinnlegg nå. Det er fra en anonym avsender som kaller seg «Fornærmet HF-gutt», og vil ta et oppgjør med deg som debattredaktør.</I>\r\n\r\n-- Oi, hvis det er personlig hets kan vi ikke trykke det. Hvor ille er det?\r\n\r\n-- <I>Det er ganske sykt. Han skriver at han har møtt deg flere ganger på fest, og at du alltid er høylytt, arrogant og snakker for mye.</I>\r\n\r\n-- Hahaha, hyggelig da. Men vi kan jo ikke trykke det når 

In [27]:

Story._meta.get_field('modified').pre_save = pre_save

for story in Story.objects.filter(publication_status__lt=Story.STATUS_PUBLISHED):
    data = old_api.get(story.pk)
    created = parser(data.get('dato'))
    modified = parser(data.get('version_date'))
    story.created = created
    story.modified = modified
    story.publication_status = flags[data.get('flagg')]
    story.save(update_fields=['created', 'modified'] )
    print(story)
    print(created, modified, story.modified, sep='\n')

NameError: name 'parser' is not defined

In [ ]:
import pytz
pytz.country_timezones('no')

In [ ]:
def get_story_type(prodsys_mappe):
    """ Find correct story type for the story based on legacy label. """
    try:
        story_type = StoryType.objects.filter(prodsys_mappe=prodsys_mappe)[0]
    except IndexError:
        generic_section, exists = Section.objects.get_or_create(
            title='New Section')
        story_type = StoryType.objects.create(
            prodsys_mappe=prodsys_mappe,
            name="New Story Type ({label})".format(label=prodsys_mappe),
            section=generic_section,
        )
    # finally:
    return story_type

In [ ]:
Story.objects.filter(publication_status__lte=1).delete()

In [ ]:
response = api.post(id=23215, data={'arbeidstittel': 'awesome', 'produsert': 0})
print(response.request.headers, response.request.body, sep='\n')
print(response.url, response.status_code, response.text)
testsak = api.get(23215)
print(testsak['arbeidstittel'], testsak['version_no'], testsak['produsert'])

In [ ]:
from django.contrib.auth.models import Permission
pp = Permission.objects.filter(codename__contains='_story')
for p in pp: print(p.codename)jjj

In [ ]:
from django.contrib.auth import authenticate
create_user = get_user_model().objects.create
get_user_model().objects.filter(username='api').delete()
api_user = get_user_model().objects.create_user(
    username='api',
    password='api',
    email='api@api.com',
)
print(api_user)
login = authenticate(username='api', password='api')
print(login)

In [ ]:
from django.contrib.auth import get_user_model

def api_user():
    User = get_user_model()
    try:
        user = User.objects.get(username='api')
    except User.DoesNotExist:
        user = User.objects.create_user(
            username='api',
            password='api',
            email='api@example.com',
        )
    permissions = list(Permission.objects.filter(codename__contains='_story'))
    user.user_permissions.add(*permissions)
    for p in permissions:
        print(p.codename)
    return user

api_user()

In [ ]:
saker = api.list()
til_web = list(filter(lambda sak: sak['produsert'] == '6', saker))
pprint(til_web[7])


In [ ]:
pprint(api.get(23194))

In [ ]:
def deserialize_story(story):
    output = [] 
    head_tags = [('tit', 'title'),('ing', 'lede'),('tema', 'theme_word'),('stikktit', 'kicker')]
    for tag, attr in head_tags:
        text = getattr(story, attr)
        if text:
            output.append(f'@{tag}:{text}')
    for bl in story.byline_set.select_related('contributor'):
        output.append(str(bl))
    output.append(story.bodytext_markup) 
    for aside in story.asides():
        output.append(aside.child.bodytext_markup)
    for pullquote in story.pullquotes():
        output.append(pullquote.child.bodytext_markup)
    return '\n'.join(output)
    
story = Story.objects.order_by('?').last()
txt1 = deserialize_story(story)
prodstory = api.get(story.prodsak_id)
txt2 = prodstory.get('tekst')
print(story.prodsak_id)
print(txt1)
print(txt2)

In [ ]:
story = Story.objects.get(id=61090)
story.images()

In [ ]:
imgstories = Story.objects.filter(pk__in=StoryImage.objects.all().values_list('parent_story', flat=True).distinct())
imgstories[0].pk
